set directory first

In [50]:
import os
%pwd

'e:\\'

In [51]:
os.chdir('../')

In [52]:
%pwd

'e:\\'

'e:\\projects_final file\\Text  Summarization\\Text-Summarization-Project'

####  WorkFlow

1.Update Config.yaml file

2.Update params.yaml file

3.Update entity

4.Update configuration manager in src config

5.Update the components

6.Update the pipeline

7.Update the main.py

8.Update the app.py


1.Update Config.yaml file

data_trasformation:


  root_dir: artifacts/data_transformation
  
  data_path: artifacts/data_transformation/samsum_dataset

  tokenizer_name: google/pegasus-cnn_dailymail 

2.Skip params for this task

3.Update entity

In [53]:
from dataclasses import dataclass
from pathlib import Path

#update modular part for DaTatransformation
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path 
    

4.Update configuration manager in src config

In [54]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [55]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config



5.Update the components


In [56]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [57]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config #data ingestion config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name) # to initialize tokenizer
        
        
    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))     

6.Update the pipeline

In [58]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'

7.Update the main.py